<a href="https://colab.research.google.com/github/marco-siino/fake_news_spreaders_detection/blob/main/FNS_CNN_MSiino_ModelNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset.
CNN Model, Training and Testing Notebook.
Code by M. Siino.

From the paper: "Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset." by M.Siino et al.

## Importing modules.

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers import TextVectorization
from io import open
from pathlib import Path

## Importing DS and extract in current working directory.

In [ ]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-training-2020-02-23.zip"
urlTestSet="https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-test-2020-02-23.zip"

training_set = tf.keras.utils.get_file("pan20-author-profiling-training-2020-02-23.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan20-author-profiling-test-2020-02-23.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan20-author-profiling-training-2020-02-23')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan20-author-profiling-test-2020-02-23')
os.rename('pan20-author-profiling-training-2020-02-23_extracted', 'pan20-author-profiling-training-2020-02-23')
os.rename('pan20-author-profiling-test-2020-02-23_extracted', 'pan20-author-profiling-test-2020-02-23')
print(training_set)
print(training_set_dir)

!ls -A

2146304/2135236 [==============================] - 0s 0us/step
./pan20-author-profiling-training-2020-02-23.zip
./pan20-author-profiling-training-2020-02-23
.config
__MACOSX
pan20-author-profiling-test-2020-02-23
pan20-author-profiling-test-2020-02-23.zip
pan20-author-profiling-training-2020-02-23
pan20-author-profiling-training-2020-02-23.zip
sample_data


## Build folders hierarchy to use Keras folders preprocessing function.

In [ ]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')
if not os.path.exists('train_dir_es'):
    os.makedirs('train_dir_es')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_es/0'):
    os.makedirs('train_dir_es/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')
if not os.path.exists('train_dir_es/1'):
    os.makedirs('train_dir_es/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')
if not os.path.exists('test_dir_es'):
    os.makedirs('test_dir_es')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_es/0'):
    os.makedirs('test_dir_es/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')
if not os.path.exists('test_dir_es/1'):
    os.makedirs('test_dir_es/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

.config						sample_data
__MACOSX					test_dir_en
pan20-author-profiling-test-2020-02-23		test_dir_es
pan20-author-profiling-test-2020-02-23.zip	train_dir_en
pan20-author-profiling-training-2020-02-23	train_dir_es
pan20-author-profiling-training-2020-02-23.zip


## Set language and directory paths.


In [ ]:
# Set en and es train_dir and test_dir paths.
language='es'

truth_file_training_dir_es=training_set_dir+'/'+language+'/'
truth_file_training_path_es = truth_file_training_dir_es+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_es = truth_file_test_dir+'/'+language+'.txt'


language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_path_en = truth_file_test_dir+'/'+language+'.txt'

## Read truth.txt to organize training dataset folders.



In [ ]:
# Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

language='es'
# Organize ES folders.
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_es, "r")
# use readline() to read the first line
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_es+fNameXml):
      os.rename(truth_file_training_dir_es+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Read truth.txt to organize test dataset folders.

In [ ]:
#Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

#Organize EN folders.
language='es'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_es, "r")
# use readline() to read the first line
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Function to pre-process source text.

In [ ]:
def custom_standardization(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_es_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="es">', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_author_lang_es_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

## Building the dataset.

In [ ]:
batch_size=1

# Build the dataset for Spanish.
language='es'

raw_train_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language,
    batch_size=batch_size,
    #validation_split=0.0,
    #subset='training',
    shuffle='false',
    seed=1
    )

raw_test_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language,
    batch_size=batch_size,
    shuffle='false'
    )


# Build the dataset for Spanish.
language='en'

raw_train_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language,
    batch_size=batch_size,
    #validation_split=0.0,
    #subset='training',
    shuffle='false',
    seed=1
    )

raw_test_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language,
    batch_size=batch_size,
    shuffle='false'
    )


Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


## First model's layer: Text Vectorization.

In [ ]:
# Maximum number of words allowed 80000 in our dictionary.
max_features = 80000
# After tokenization 4500 covers all the document lenghts in our dataset.
sequence_length = 4500

vectorize_layer_es = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer_en = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = raw_train_ds_en.map(lambda x, y: x)
vectorize_layer_en.adapt(train_text)

train_text = raw_train_ds_es.map(lambda x, y: x)
vectorize_layer_es.adapt(train_text)

## Some training parameters...

In [ ]:
# Word embedding dimensions.
embedding_dim = 500

num_runs = 5
num_epochs_per_run = 100

opt = tf.keras.optimizers.RMSprop()

## Training and evaluation of English model

In [ ]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
    epochs_accuracy=[]
    model_en = tf.keras.Sequential([
                                    tf.keras.Input(shape=(1,), dtype=tf.string),
                                    vectorize_layer_en,
                                    layers.Embedding(max_features + 1, embedding_dim),
                                    layers.Dropout(0.8),

                                    layers.Conv1D(256,16,activation='relu'),
                                    layers.MaxPooling1D(),
                                    layers.Dropout(0.6),

                                    layers.Dense(512,activation='relu'),

                                    layers.GlobalAveragePooling1D(),
                                    layers.Dropout(0.2),
                                    layers.Dense(1)
    ])
    model_en.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer=opt, metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.0)])

    for epoch in range (0,num_epochs_per_run):
        history = model_en.fit(
          raw_train_ds_en,
          validation_data = raw_test_ds_en,
          epochs=1,
          shuffle=False,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )
        accuracy = history.history['val_binary_accuracy']
        print("Run: ",run,"/ Accuracy at epoch ",epoch," is: ", accuracy[0],"\n")
        epochs_accuracy.append(accuracy[0])

    print("Accuracies over epochs:",epochs_accuracy,"\n\n")
    runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on English are:", runs_accuracy)
print("The median for English is:",runs_accuracy[2],"\n\n\n")


300/300 [==============================] - 23s 72ms/step - loss: 0.7199 - binary_accuracy: 0.4500 - val_loss: 0.7947 - val_binary_accuracy: 0.5400
Run:  1 / Accuracy at epoch  0  is:  0.5400000214576721 

300/300 [==============================] - 21s 71ms/step - loss: 0.6653 - binary_accuracy: 0.6200 - val_loss: 0.8889 - val_binary_accuracy: 0.6050
Run:  1 / Accuracy at epoch  1  is:  0.6050000190734863 

300/300 [==============================] - 21s 70ms/step - loss: 0.4493 - binary_accuracy: 0.7833 - val_loss: 0.9558 - val_binary_accuracy: 0.6800
Run:  1 / Accuracy at epoch  2  is:  0.6800000071525574 

300/300 [==============================] - 21s 70ms/step - loss: 0.2501 - binary_accuracy: 0.9000 - val_loss: 1.4367 - val_binary_accuracy: 0.6850
Run:  1 / Accuracy at epoch  3  is:  0.6850000023841858 

300/300 [==============================] - 21s 70ms/step - loss: 0.1460 - binary_accuracy: 0.9467 - val_loss: 1.9112 - val_binary_accuracy: 0.6450
Run:  1 / Accuracy at epoch  4  i

## Training and evaluation of Spanish model

In [ ]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
    epochs_accuracy=[]
    model_es = tf.keras.Sequential([
                                    tf.keras.Input(shape=(1,), dtype=tf.string),
                                    vectorize_layer_es,
                                    layers.Embedding(max_features + 1, embedding_dim),
                                    layers.Dropout(0.8),

                                    layers.Conv1D(256,16,activation='relu'),
                                    layers.MaxPooling1D(),
                                    layers.Dropout(0.6),

                                    layers.Dense(512,activation='relu'),

                                    layers.GlobalAveragePooling1D(),
                                    layers.Dropout(0.2),
                                    layers.Dense(1)
    ])
    model_es.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer=opt, metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

    for epoch in range (0,num_epochs_per_run):
        history = model_es.fit(
          raw_train_ds_es,
          validation_data = raw_test_ds_es,
          epochs=1,
          shuffle=False,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )
        accuracy = history.history['val_binary_accuracy']
        print("Run: ",run,"/ Accuracy at epoch ",epoch," is: ", accuracy[0],"\n")
        epochs_accuracy.append(accuracy[0])

    print("Accuracies over epochs:",epochs_accuracy,"\n\n")
    runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on Spanish are:", runs_accuracy)
print("The median for Spanish is:",runs_accuracy[2],"\n\n\n")


300/300 [==============================] - 7s 22ms/step - loss: 0.6955 - binary_accuracy: 0.6133 - val_loss: 0.6104 - val_binary_accuracy: 0.6950
Run:  1 / Accuracy at epoch  0  is:  0.6949999928474426 

300/300 [==============================] - 7s 22ms/step - loss: 0.5337 - binary_accuracy: 0.7700 - val_loss: 0.5446 - val_binary_accuracy: 0.6950
Run:  1 / Accuracy at epoch  1  is:  0.6949999928474426 

300/300 [==============================] - 6s 21ms/step - loss: 0.3640 - binary_accuracy: 0.8700 - val_loss: 0.5696 - val_binary_accuracy: 0.7800
Run:  1 / Accuracy at epoch  2  is:  0.7799999713897705 

300/300 [==============================] - 6s 22ms/step - loss: 0.1818 - binary_accuracy: 0.9267 - val_loss: 0.8202 - val_binary_accuracy: 0.7800
Run:  1 / Accuracy at epoch  3  is:  0.7799999713897705 

300/300 [==============================] - 7s 23ms/step - loss: 0.0799 - binary_accuracy: 0.9833 - val_loss: 0.9917 - val_binary_accuracy: 0.7550
Run:  1 / Accuracy at epoch  4  is:  0